# San Joaquin Valley Township-Range Well Completion Reports Datasets

Related links:
* For the documentation about this dataset, its source, how to download, and the features of interest, please refer to our [Well Completion Reports Dataset](/doc/assets/well_completion_reports.md) documentation.
* For the explanations on how the sortage mapping datasets are mapped to TownshipRange please refer to our [Public Land Survey System](../assets/plss_sanjoaquin_riverbasin.md) documentation.

In [16]:
import sys
sys.path.append('..')

In [17]:
import numpy as np
import altair as alt
from lib.well_completion import WellCompletionReportsDataset
from lib.viz import draw_missing_data_chart, get_base_map, view_year_with_slider, view_year_side_by_side, display_data_on_map, visualize_seasonality_by_month

In [18]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
wcr = WellCompletionReportsDataset()

Pre-process the well completion report data, geospatial location and additional elevation, combine them into the the geospatial map_df dataset and overlay the San Joaquin Valley boundaries to keep only the data in the San Joaquin Valley

In [ ]:
wcr.preprocess_map_df(features_to_keep=["WCRNUMBER", "YEAR", "MONTH", "BOTTOMOFPERFORATEDINTERVAL", "GROUNDSURFACEELEVATION", "STATICWATERLEVEL", "TOPOFPERFORATEDINTERVAL", "TOTALDRILLDEPTH", "TOTALCOMPLETEDDEPTH", "WELLYIELD", "USE", "geometry"])
wcr.keep_only_sjv_data()
wcr.map_df

Look at missing data in the dataset

In [ ]:
draw_missing_data_chart(wcr.map_df)

Map of the wells completed in the year 2021 in the San Joaquin Valley

In [ ]:
display_data_on_map(wcr.map_df, feature="USE", year=2021)

Check for seasonality in well completion

In [ ]:
visualize_seasonality_by_month(wcr.get_well_count_by_year_month(), feature="WELL_COUNT")

Next, for all Township-Ranges and every year:
1. Compute the average well value in every Township-Range
2. Count the number of wells
3. Fill NaN values with 0

In [ ]:
wcr.compute_features_by_township(features_to_average=["BOTTOMOFPERFORATEDINTERVAL", "GROUNDSURFACEELEVATION", "STATICWATERLEVEL", "TOPOFPERFORATEDINTERVAL", "TOTALDRILLDEPTH", "TOTALCOMPLETEDDEPTH", "WELLYIELD"], add_well_count=True, fill_na_with_zero=False)
wcr.map_df

Fill in missing Township-Range with:
1. a WELL_COUNT of 0
2. NaN for the features

In [ ]:
wcr.fill_townships_with_no_data(features_to_fill=["WELL_COUNT", "WELL_COUNT_AGRICULTURE", "WELL_COUNT_DOMESTIC", "WELL_COUNT_INDUSTRIAL", "WELL_COUNT_PUBLIC"], feature_value=0)
wcr.fill_townships_with_no_data(features_to_fill=["BOTTOMOFPERFORATEDINTERVAL_AVG", "GROUNDSURFACEELEVATION_AVG", "STATICWATERLEVEL_AVG", "TOPOFPERFORATEDINTERVAL_AVG",
                                                  "TOTALDRILLDEPTH_AVG", "TOTALCOMPLETEDDEPTH_AVG", "WELLYIELD_AVG"], feature_value=np.nan)
wcr.map_df

In [ ]:
wcr.prepare_output_from_map_df()
wcr.output_dataset_to_csv(output_filename="../assets/outputs/well_completion_reports.csv")

## Visualizing the Normalized Well Count


Compute the WELL_COUNT normalized by year

In [ ]:
normalized_df = wcr.return_yearly_normalized_township_feature(feature_name="WELL_COUNT", normalize_method = "minmax")
normalized_df

### Comparison per year

In [ ]:
view_year_side_by_side(normalized_df, feature="WELL_COUNT_NORMALIZED", title="Well Counts")

### Map visualization for 2018

In [ ]:
display_data_on_map(normalized_df, feature="WELL_COUNT_NORMALIZED", year=2018)

### Map visualization for 2021

In [ ]:
display_data_on_map(normalized_df, feature="WELL_COUNT_NORMALIZED", year=2021)